In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
import os

base_dir = './'
train_dir = os.path.join(base_dir, '../dataset/processed')
validation_dir = os.path.join(base_dir, '../dataset/test')


In [3]:
batch_size = 2
total_train = 188
total_test = 24

In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        horizontal_flip=True,
        shear_range = 0.2,
        fill_mode='nearest'
      )

test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 188 images belonging to 5 classes.
Found 24 images belonging to 5 classes.


In [6]:
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),

        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),

        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(5, activation='sigmoid')
    ])

    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
  
    return model

In [7]:
checkpoint_path = './trained_model/model.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

In [8]:
model = create_model()

model.save_weights(checkpoint_path.format(epoch=0))

model.fit(
      train_generator,
      epochs=20,
      steps_per_epoch=total_train//batch_size,
      validation_data=validation_generator,
      validation_steps=total_test//batch_size, 
      callbacks=[cp_callback]
      )

Epoch 1/20
42/42 [==============================] - ETA: 0s - loss: 1.5279 - accuracy: 0.5119
Epoch 1: saving model to ./trained_model\model.ckpt
42/42 [==============================] - 7s 144ms/step - loss: 1.5279 - accuracy: 0.5119 - val_loss: 1.3910 - val_accuracy: 0.5000
Epoch 2/20
42/42 [==============================] - ETA: 0s - loss: 1.2386 - accuracy: 0.6071
Epoch 2: saving model to ./trained_model\model.ckpt
42/42 [==============================] - 7s 178ms/step - loss: 1.2386 - accuracy: 0.6071 - val_loss: 1.4354 - val_accuracy: 0.4286
Epoch 3/20
42/42 [==============================] - ETA: 0s - loss: 1.3069 - accuracy: 0.5714
Epoch 3: saving model to ./trained_model\model.ckpt
42/42 [==============================] - 7s 158ms/step - loss: 1.3069 - accuracy: 0.5714 - val_loss: 1.2062 - val_accuracy: 0.5000
Epoch 4/20
42/42 [==============================] - ETA: 0s - loss: 0.9731 - accuracy: 0.6786
Epoch 4: saving model to ./trained_model\model.ckpt
42/42 [================

In [9]:
model.save_weights('./trained_model/weights.h5')
model.save('./trained_model/model.h5')